In [78]:
import pandas as pd
import numpy as np
import nltk
import string

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

In [57]:
#lyric files
eurovision = open("data/intersection.txt", "r")
c60s = open("data/1960s.txt", "r")
c70s = open("data/1970s.txt", "r")
c80s = open("data/1980s.txt", "r")
c90s = open("data/1990s.txt", "r")
c00s = open("data/2000s.txt", "r")
c10s = open("data/2010s.txt", "r")

eurovisionLyrics = eurovision.read()
lyrics60s = c60s.read()
lyrics70s = c70s.read()
lyrics80s = c80s.read()
lyrics90s = c90s.read()
lyrics00s = c00s.read()
lyrics10s = c10s.read()

eurovision.close()
c60s.close()
c70s.close()
c80s.close()
c90s.close()
c00s.close()
c10s.close()

In [58]:
#make lowercase
def convLower(text):
    text = text.lower()
    return text

eurovisionLyrics = convLower(eurovisionLyrics)
lyrics60s = convLower(lyrics60s)
lyrics70s = convLower(lyrics70s)
lyrics80s = convLower(lyrics80s)
lyrics90s = convLower(lyrics90s)
lyrics00s = convLower(lyrics00s)
lyrics10s = convLower(lyrics10s)

In [59]:
#remove punctuation - this doesnt remove apostrophes
def removePunctuation(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text

eurovisionLyrics = removePunctuation(eurovisionLyrics)
lyrics60s = removePunctuation(lyrics60s)
lyrics70s = removePunctuation(lyrics70s)
lyrics80s = removePunctuation(lyrics80s)
lyrics90s = removePunctuation(lyrics90s)
lyrics00s = removePunctuation(lyrics00s)
lyrics10s = removePunctuation(lyrics10s)

In [81]:
#word tokenize
wordsEurovision = word_tokenize(eurovisionLyrics)
words60s = word_tokenize(lyrics60s)
words70s = word_tokenize(lyrics70s)
words80s = word_tokenize(lyrics80s)
words90s = word_tokenize(lyrics90s)
words00s = word_tokenize(lyrics00s)
words10s = word_tokenize(lyrics10s)

In [97]:
#sentence tokenize
senEurovision = sent_tokenize(eurovisionLyrics)
sen60s = sent_tokenize(lyrics60s)
sen70s = sent_tokenize(lyrics70s)
sen80s = sent_tokenize(lyrics80s)
sen90s = sent_tokenize(lyrics90s)
sen00s = sent_tokenize(lyrics00s)
sen10s = sent_tokenize(lyrics10s)

In [94]:
#test frequency dist
words10sFD = FreqDist(words10s)
words10sFreqDist = pd.DataFrame(list(words10sFD.items()), columns = ["Word","Frequency"]) 

#words10sFD.most_common(50)

wordsEurovisionFD = FreqDist(wordsEurovision)
wordsEurovisionFreqDist = pd.DataFrame(list(wordsEurovisionFD.items()), columns = ["Word","Frequency"]) 

#wordsEurovisionFD.most_common(50)

In [104]:
#bigrams 2010s
bigrams10s = nltk.bigrams(words10s)
filtered_bigrams = [ (w1, w2) for w1, w2 in bigrams10s if len(w1) >=5 and len(w2) >= 5 ]
eng_bifreq = nltk.FreqDist(filtered_bigrams)
eng_bifreq.most_common(3)

[(('thunder', 'thunder'), 49),
 (('dohdohdoh', 'dohdoh'), 24),
 (('dohdohdoh', 'dohdohdoh'), 23)]

In [107]:
#bigrams 2000s
bigrams00s = nltk.bigrams(words00s)
filtered_bigrams = [ (w1, w2) for w1, w2 in bigrams00s if len(w1) >=5 and len(w2) >= 5 ]
eng_bifreq = nltk.FreqDist(filtered_bigrams)
eng_bifreq.most_common(3)

[(('think', 'youre'), 40), (('youre', 'ready'), 38), (('music', 'music'), 27)]

In [111]:
#bigrams 1990s
bigrams90s = nltk.bigrams(words90s)
filtered_bigrams = [ (w1, w2) for w1, w2 in bigrams90s if len(w1) >=5 and len(w2) >= 5 ]
eng_bifreq = nltk.FreqDist(filtered_bigrams)
eng_bifreq.most_common(3)

[(('mercy', 'mercy'), 28),
 (('strong', 'enough'), 23),
 (('youre', 'still'), 21)]

In [110]:
#bigrams 1980s
bigrams80s = nltk.bigrams(words80s)
filtered_bigrams = [ (w1, w2) for w1, w2 in bigrams80s if len(w1) >=5 and len(w2) >= 5 ]
eng_bifreq = nltk.FreqDist(filtered_bigrams)
eng_bifreq.most_common(3)

[(('louie', 'louie'), 22),
 (('karma', 'karma'), 16),
 (('invisible', 'touch'), 16)]

In [109]:
#bigrams 1970s
bigrams70s = nltk.bigrams(words70s)
filtered_bigrams = [ (w1, w2) for w1, w2 in bigrams70s if len(w1) >=5 and len(w2) >= 5 ]
eng_bifreq = nltk.FreqDist(filtered_bigrams)
eng_bifreq.most_common(3)

[(('stayin', 'alive'), 25),
 (('doodoo', 'doodoodoo'), 20),
 (('really', 'hurts'), 18)]

In [108]:
#bigrams 1960s
bigrams60s = nltk.bigrams(words60s)
filtered_bigrams = [ (w1, w2) for w1, w2 in bigrams60s if len(w1) >=5 and len(w2) >= 5 ]
eng_bifreq = nltk.FreqDist(filtered_bigrams)
eng_bifreq.most_common(3)

[(('first', 'communion'), 41),
 (('talking', 'about'), 28),
 (('bouncy', 'bouncy'), 26)]

In [106]:
#bigrams Eurovision
bigramsEurovision = nltk.bigrams(wordsEurovision)
filtered_bigrams = [ (w1, w2) for w1, w2 in bigramsEurovision if len(w1) >=5 and len(w2) >= 5 ]
eng_bifreq = nltk.FreqDist(filtered_bigrams)
eng_bifreq.most_common(3)

[(('these', 'words'), 10), (('every', 'night'), 10), (('mamma', 'mamma'), 10)]